## 비동기 방식 크롤링

In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
URL = 'http://www.cine21.com/rank/boxoffice/domestic'

In [4]:
response = requests.get(URL)

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
movies = soup.select('.boxoffice_li')

In [9]:
movies

[]

In [10]:
soup.select_one('#boxoffice_list_content')

<div id="boxoffice_list_content">
</div>

## 웹 API 서버(Backend)에 요청 

In [11]:
URL = 'http://www.cine21.com/rank/boxoffice/domestic_content'

In [12]:
data = {
    'page': '7',
    'genre': 'all',
    'period': '2023-02-06'
}

In [22]:
response = requests.post(URL, data=data)

In [23]:
response.status_code

200

In [18]:
# print(response.text) -> 구조가 html 문서같다..? soup 객체로 변환 !
soup = BeautifulSoup(response.text, 'html.parser')

In [24]:
# 내가 가져오고 싶은 영화가 있는지 확인
movies = soup.select('.boxoffice_li')

In [26]:
movies[0].select_one('.mov_name').text

'릴리 슈슈의 모든 것'

## JSON

In [27]:
URL = 'https://jsonplaceholder.typicode.com/posts'

In [28]:
response = requests.get(URL)

In [51]:
# print(response.text)
type(response.text)

str

### JSON 문자열을 파이썬 list, dict 객체로 변환

In [33]:
import json

In [39]:
json_result = json.loads(response.text)

In [40]:
type(json_result)

list

In [ ]:
json_result

In [49]:
type(response.json())

list

## 실전 01

- 디자인 정글 크롤링
- 제목, 카테고리
- 스크롤을 통해 나오는 추가 데이터 크롤링

In [52]:
URL = 'https://www.jungle.co.kr'

In [53]:
response = requests.get(URL)

In [54]:
response

<Response [200]>

In [71]:
# 스크롤 했을 때 나오는 추가데이터 ([Network] -> Fetch/XHR 탭에서 나오는 recent.json의 요청 URL)
JUNGLE_URL = 'https://www.jungle.co.kr/recent.json?magazineOffset={}&contestOffset={}&exhibitOffset={}&galleryOffset={}'.format(
    0, 0, 0, 0
)

URL = 'https://www.jungle.co.kr/recent.json'
# # ? 뒤의 query parameters 분리
# parameters = '?magazineOffset=0   &   contestOffset=0   &   exhibitOffset=0   &   galleryOffset=0'

In [72]:
JUNGLE_URL

'https://www.jungle.co.kr/recent.json?magazineOffset=0&contestOffset=0&exhibitOffset=0&galleryOffset=0'

In [67]:
params = {
    'magazineOffset': 0,
    'contestOffset': 0,
    'exhibitOffset': 0,
    'galleryOffset': 0
}

In [68]:
response = requests.get(URL, params=params)

In [69]:
response.status_code

200

In [76]:
json.dumps({
    'key': 0,
    'key2': 3
})

'{"key": 0, "key2": 3}'

In [73]:
# json포맷의 데이터를 파이썬 list나 dict객체로 변환
import json
json_result = json.loads(response.text)

In [80]:
jungle_list = []

for data in json_result['moreList']:
    # print(data['title']) # 제목
    # print(data['targetCode']) # 카테고리
    jungle_list.append({
        '제목': data['title'],
        '카테고리': data['targetCode']
    })
jungle_list

[{'제목': '자몽 아카데미 키즈 크리에이터 콘텐츠 스쿨', '카테고리': '공모전'},
 {'제목': '2023년 대한민국 청년정책 공모전', '카테고리': '공모전'},
 {'제목': '[궁금한 인터뷰] 건축으로 표현하는 일상의 위대함', '카테고리': '매거진'},
 {'제목': '[정글 포커스] 환경에 진심인 기업', '카테고리': '매거진'},
 {'제목': '[온라인 활동] 초보자도 할 수 있는 펀딩 프로그램 모집', '카테고리': '공모전'},
 {'제목': '[온라인 활동] 상품, APP, WEB, 서비스 기획직무를 함께 스터디해요!!', '카테고리': '공모전'}]

In [ ]:
# 마지막 페이지까지 크롤링
params = {
    'magazineOffset': 0,
    'contestOffset': 0,
    'exhibitOffset': 0,
    'galleryOffset': 0
}

jungle_list = []
for i in range(100):
    print('현재 파라미터', params)
    response = requests.get(URL, params=params)

    json_result = json.loads(response.text)

    for data in json_result['moreList']:
        # print(data['title']) # 제목
        # print(data['targetCode']) # 카테고리
        jungle_list.append({
            '제목': data['title'],
            '카테고리': data['targetCode']
        })
    
    # next parameter 값
    for key in params:
        # print(key, json_result[key])
        params[key] = json_result[key]
    # print('다음 파라미터?', params)

    if json_result['existMore'] == False:
        break


In [ ]:
jungle_list

## 실전 02

- 로켓펀치 사이트 크롤링
- 회사명, 회사설명, 채용공고 (회사 별 여러 개)
- 총 10 페이지 크롤링

In [ ]:
# URL = 'https://www.rocketpunch.com/jobs'

In [99]:
URL = 'https://www.rocketpunch.com/api/jobs/template'
parameters = '?page=2  &  q='

In [100]:
params = {
    'page': 1,
    'q': '',
}

In [101]:
response = requests.get(URL, params=params)

In [102]:
response.status_code

200

In [103]:
# '{'로 시작하는 문자열임으로, 
# 파이썬 딕셔너리 객체로 만들기 위해 json.loads('문자열') 메서드 사용
json_result = json.loads(response.text)

In [104]:
# json 안에 html 문서가 들어가있는 경우! SEO를 위해 이런식으로 서버를 구축하기도 함
soup = BeautifulSoup(json_result['data']['template'], 'html.parser')

In [105]:
company_list = soup.select('.company-list > .company.item')

In [ ]:
rocket_list = []
for company in company_list:
    company_name = company.select_one('.company-name strong').text # 회사명
    company_desc = company.select_one('.description').text # 회사 설명

    # 채용공고들 (여러 개)
    job_details = company.select('.job-detail')

    jobs = []
    for job_detail in job_details:
        a_tag = job_detail.select_one('a')
        a_tag.text # 채용공고명
        a_tag.attrs['href'] # 채용공고 링크
        jobs.append({
            '공고명': a_tag.text,
            '링크': a_tag.attrs['href']
        })
    
    rocket_list.append({
        '회사명': company_name,
        '회사설명': company_desc,
        '채용공고': jobs
    })

rocket_list

In [109]:
rocket_list = []

for page in range(1, 11):
    # 한 페이지씩 넘기는 parameter 생성
    params = {
        'page': page,
        'q': '',
    }
    print(f'현재 {page}페이지 진행 중...')

    # 위에서 한 페이지 작업할 때 했던 코드들 가져오기
    response = requests.get(URL, params=params)
    json_result = json.loads(response.text) # 1차적으로 json 데이터를 파이썬 객체로 변경
    soup = BeautifulSoup(json_result['data']['template'], 'html.parser') # 그 안에 html을 soup 객체로 만듬

    company_list = soup.select('.company-list > .company.item')
    
    for company in company_list:
        company_name = company.select_one('.company-name strong').text # 회사명
        company_desc = company.select_one('.description').text # 회사 설명

        # 채용공고들 (여러 개)
        job_details = company.select('.job-detail')

        jobs = []
        for job_detail in job_details:
            a_tag = job_detail.select_one('a')
            a_tag.text # 채용공고명
            a_tag.attrs['href'] # 채용공고 링크
            jobs.append({
                '공고명': a_tag.text,
                '링크': a_tag.attrs['href']
            })
        
        rocket_list.append({
            '회사명': company_name,
            '회사설명': company_desc,
            '채용공고': jobs
        })

print(len(rocket_list))

현재 1 진행 중...
현재 2 진행 중...
현재 3 진행 중...
현재 4 진행 중...
현재 5 진행 중...
현재 6 진행 중...
현재 7 진행 중...
현재 8 진행 중...
현재 9 진행 중...
현재 10 진행 중...
200


In [110]:
# 함수화

URL = 'https://www.rocketpunch.com/api/jobs/template'
def rocket_crawler(params):
    '''
    로켓펀치 채용공고를 크롤링하는 함수입니다.
    params : dict (page, q)
    return : list (채용공고)
    '''
    response = requests.get(URL, params=params)
    json_result = json.loads(response.text)
    soup = BeautifulSoup(json_result['data']['template'], 'html.parser')

    company_list = soup.select('.company-list > .company.item')
    
    results = []
    for company in company_list:
        company_name = company.select_one('.company-name strong').text # 회사명
        company_desc = company.select_one('.description').text # 회사 설명

        # 채용공고들 (여러 개)
        job_details = company.select('.job-detail')

        jobs = []
        for job_detail in job_details:
            a_tag = job_detail.select_one('a')
            a_tag.text # 채용공고명
            a_tag.attrs['href'] # 채용공고 링크
            jobs.append({
                '공고명': a_tag.text,
                '링크': a_tag.attrs['href']
            })
        
        results.append({
            '회사명': company_name,
            '회사설명': company_desc,
            '채용공고': jobs
        })
    
    return results

In [ ]:
# 함수를 실행할 부분
rocket_list = []

for page in range(1, 11):
    # 한 페이지씩 넘기는 parameter 생성
    params = {
        'page': page,
        'q': '',
    }
    print(f'현재 {page}페이지 진행 중...')

    rocket_list.extend(rocket_crawler(params))

rocket_list

## 실전 03

In [188]:
# URL = 'https://finance.naver.com/item/sise.naver?code=005930'

URL = 'https://finance.naver.com/item/sise_day.naver'
parameters = '?code=005930&page=4'

params = {
    'code': '005930',
    'page': 1
}

In [121]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

In [189]:
res = requests.get(URL, params=params, headers=headers)
res.status_code

200

In [190]:
soup = BeautifulSoup(res.text, 'html.parser')

In [191]:
tr_tags = soup.select('tr')

In [192]:
column_names = [th.text for th in tr_tags[0].select('th')]

In [193]:
column_names

['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']

In [194]:
# 1차적으로 앞의 2줄을 자름
tr_tags = tr_tags[2:]

In [195]:
# 2차적으로 5개, 5개씩 가져옴
tr_tags = tr_tags[:5] + tr_tags[8:-2]

In [196]:
stock_list = []
for tr_tag in tr_tags:
    td_tags = tr_tag.select('td')
    stock = [td_tag.text.strip() for td_tag in td_tags]
    stock_list.append(stock)

In [197]:
stock_list

[['2023.02.13', '62,900', '100', '62,900', '63,000', '62,300', '10,679,961'],
 ['2023.02.10', '62,800', '200', '62,600', '63,000', '62,400', '9,382,576'],
 ['2023.02.09', '63,000', '100', '63,000', '63,300', '62,300', '12,164,865'],
 ['2023.02.08', '63,100', '1,200', '62,800', '63,300', '62,400', '12,243,125'],
 ['2023.02.07', '61,900', '300', '61,900', '62,500', '61,600', '14,491,039'],
 ['2023.02.06', '61,600', '2,200', '62,800', '63,000', '61,600', '15,529,356'],
 ['2023.02.03', '63,800', '300', '63,900', '64,000', '63,000', '15,194,598'],
 ['2023.02.02', '63,500', '1,700', '63,200', '63,900', '62,600', '23,285,983'],
 ['2023.02.01', '61,800', '800', '62,600', '62,700', '61,000', '18,570,133'],
 ['2023.01.31', '61,000', '2,300', '63,500', '63,700', '61,000', '29,746,731']]

In [ ]:
stock_list = []
for page in range(1, 11):
    params = {
        'code': '005930',
        'page': page
    }

    res = requests.get(URL, params=params, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    tr_tags = soup.select('tr')
    tr_tags = tr_tags[2:7] + tr_tags[10:-2]
    
    for tr_tag in tr_tags:
        td_tags = tr_tag.select('td')
        stock = [td_tag.text.strip() for td_tag in td_tags]
        stock_list.append(stock)
    print(stock_list[-1])

In [173]:
final_result = [column_names] + stock_list

In [175]:
with open('./samsung_stock.tsv', 'w', encoding='utf-8') as f:
    f.write('\n'.join(['\t'.join(fr) for fr in final_result]))